In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

%matplotlib inline

In [2]:
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_180526.txt')

In [3]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,00:00:00,REGULAR,6625470,2245312
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,04:00:00,REGULAR,6625494,2245314
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,08:00:00,REGULAR,6625514,2245338
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,12:00:00,REGULAR,6625595,2245415
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,16:00:00,REGULAR,6625791,2245481


In [4]:
maps_api = 'SECRET API KEY'

In [5]:
import googlemaps

In [6]:
gmaps = googlemaps.Client(key=maps_api)

ValueError: Invalid API key provided.

In [7]:
stations = df['STATION'].unique()

In [8]:
ny_stations = list(map(lambda x: x + ', New York, NY', stations))

In [9]:
from collections import defaultdict

In [10]:
unique_geocodes = defaultdict(str)

In [12]:
download=False

if download==True:
    for address in ny_stations:
        geocode = gmaps.geocode(address)
        unique_geocodes[address] = geocode

IndentationError: expected an indented block (<ipython-input-12-8f9c2e3a86ac>, line 5)

In [28]:
zipcodes = []

for geocode in unique_geocodes:
    try:
        print(geocode)
        print('Successfully found zip data')
        address = geocode[0]['address_components']
        if address[len(address) - 1]['types'] == ['postal_code']:
            zipcodes.append(address[len(address)- 1]['long_name'])
    except:
        print('No zip data')

Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data
Successfully found zip data
No zip data


In [17]:
test_add = gmaps.geocode('59 ST, New York, NY')

In [31]:
json_str = json.dumps(test_add)

In [32]:
json_zip = json.loads(json_str)

In [104]:
json_zip[0]['address_components'][6]['short_name']

'10022'

## Income data

In [1]:
!ls

16zpallagi.csv        benson-KF.ipynb       benson_zipcodes.ipynb
KFtest.txt            benson.ipynb          turnstile_may26.txt


In [2]:
inc = pd.read_csv('16zpallagi.csv')

In [ ]:
inc.to_csv('income-data.csv')